# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [4]:
import os
import sys
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.environment import Environment

from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import ScriptRunConfig
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.widgets import RunDetails


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
experiment = Experiment(ws, name="mp_nn_hyper")


In [5]:
ws = Workspace.from_config()
experiment = Experiment(ws, name="mp_nn_hyper")
datastore = ws.get_default_datastore()

user_path = os.path.abspath(os.path.join('..'))
if user_path not in sys.path:
    sys.path.append(user_path + '/odl_user_167740/')
datastore.upload(src_dir='./data',
                 target_path='datasets/mpNFRO',
                 overwrite=True)

"Datastore.upload" is deprecated after version 1.0.69. Please use "Dataset.File.upload_directory" to upload your files             from a local directory and create FileDataset in single method call. See Dataset API change notice at https://aka.ms/dataset-deprecation.


Uploading an estimated of 3 files
Uploading ./data/.amlignore
Uploaded ./data/.amlignore, 1 files out of an estimated total of 3
Uploading ./data/.amlignore.amltmp
Uploaded ./data/.amlignore.amltmp, 2 files out of an estimated total of 3
Uploading ./data/micropol-removal.csv
Uploaded ./data/micropol-removal.csv, 3 files out of an estimated total of 3
Uploaded 3 files


$AZUREML_DATAREFERENCE_b548bd629354438aab6ecfca70f829b3

In [6]:
# A computer cluster is created for the script run configuration 

computer_cluster = "vic-clu"

computer_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                        min_nodes=0, 
                                                        max_nodes=2)
cluster_name = ComputeTarget.create(ws, computer_cluster, computer_config)

cluster_name.wait_for_completion(show_output=True)

InProgress........
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

The "train.py" script will make use of the MLPRegressor of the package scikit-learn. The model uses neural networks to predict a response. The data is first prepared with a onehotencoder to account for categorical data, and then is normalized with a minmaxscaler.

The estimator specifies the configuration to run the train script. It contains a path to the train.py file, the computer target and the running environment. The running environment is created with a .yml file where the env dependencies are called.

Next, the hyperdrive configuration must contain first the run configuration and the other requirement to create and run the models. Hyperparameter sampling will create the parameters out of a random paramater sampling object. The policy for early termination used is the bandit policy. The primary metric is R².


In [7]:
%%writefile conda_dependencies.yml

dependencies:
- python=3.6.2
- scikit-learn
- pandas
- pip:
  - azureml-defaults

Writing conda_dependencies.yml


In [8]:
sklearn_env = Environment.from_conda_specification(name = 'sklearn-env', file_path = './conda_dependencies.yml')

In [10]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor=0.15, evaluation_interval=2, delay_evaluation=4)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({"hidden_layer_sizes": choice(100,200),
                                          "solver": choice(['lbfgs', 'sgd']),
                                          "activation": choice(['relu', 'logistic']),
                                          "alpha": choice([0.0001,0.001]),
                                          "tol": choice(0.005,0.01),
                                          "max_iter": choice(500,1000)})

#TODO: Create your estimator and hyperdrive config

comp_target = ws.compute_targets['vic-clu']

estimator = ScriptRunConfig(source_directory=user_path+'/odl_user_167740/', script='train.py', 
                      compute_target=comp_target,
                      environment=sklearn_env)


hyperdrive_run_config = HyperDriveConfig(run_config=estimator,
hyperparameter_sampling=param_sampling,
policy=early_termination_policy,
primary_metric_name='r2_score',
primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
max_total_runs=4)



In [11]:
#TODO: Submit your experiment

hyperdrive_run = experiment.submit(hyperdrive_run_config)


## Run Details

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [12]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

{'runId': 'HD_366106aa-e985-48a2-86bf-648baf4c5e5b',
 'target': 'vic-clu',
 'status': 'Completed',
 'startTimeUtc': '2021-12-22T06:55:58.060758Z',
 'endTimeUtc': '2021-12-22T07:09:14.646719Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "r2_score", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '99f4489c-9db8-4090-882d-75b917ff2f0e',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1063-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.36.0',
  'space_size': '64',
  'score': '0.8191880832638276',
  'best_child_run_id': 'HD_366106aa-e985-48a2-86bf-648baf4c5e5b_1',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg167740.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_366106aa-e985-48a2-86bf-648baf4c5e5b/azureml-logs/

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [13]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run)

Run(Experiment: mp_nn_hyper,
Id: HD_366106aa-e985-48a2-86bf-648baf4c5e5b_1,
Type: azureml.scriptrun,
Status: Completed)


In [16]:
#TODO: Save the best model
best_run.register_model(model_name = 'bestMLP_neuralnetwork', model_path = 'outputs/')


Model(workspace=Workspace.create(name='quick-starts-ws-167740', subscription_id='610d6e37-4747-4a20-80eb-3aad70a55f43', resource_group='aml-quickstarts-167740'), name=bestMLP_neuralnetwork, id=bestMLP_neuralnetwork:1, version=1, tags={}, properties={})

## Model Deployment

I chose to deploy the AutoML model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

